In [128]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

ERROR! Session/line number was not unique in database. History logging moved to new session 31


In [129]:
df = pd.read_csv('./data/train_data.csv', low_memory=False)
df

,Unnamed: 0,Id,OpportunityId,CreatedById,CreatedDate,CreatedDateForInsert,StageName,Amount,Probability,PrevOpportunityStageUpdate,...,SCC_Interest__c,Company_s_CRM_Type__c,Deal_type__c,Needs__c,Type_of_Users__c,TAM__c,Secondary_CSM__c,Source_Type__c,Original_Lead_Source__c,Original_Lead_Source_Details__c
0,0,0081300002NRVGwAAP,00613000019fCVIAA2,00530000004M1PmAAK,6/30/2014 15:21,6/30/2014 15:21,0% probability Closed Lost,0.0,100,6/30/2014 15:21,...,0,NaN,NaN,NaN,NaN,0051B000009NLbeQAG,0051B00000CsqZ4QAJ,NaN,Inbound,Marketplace - SFDC AppExchange
1,1,0081300002OmHzxAAF,0061300001AAxCTAA1,00530000004M1PmAAK,8/27/2014 20:42,8/27/2014 20:42,100% Won / PO / Invoiced,14250.0,100,8/27/2014 20:42,...,0,Salesforce,NaN,NaN,NaN,0051300000C41JZAAZ,0051B000009MC2dQAG,NaN,Inbound,Marketplace - SFDC AppExchange
2,2,0081300002OmI2DAAV,0061300001AAxCTAA1,00530000004M1PmAAK,8/27/2014 20:42,8/27/2014 20:43,100% Won / PO / Invoiced,14250.0,30,8/27/2014 20:42,...,0,Salesforce,NaN,NaN,NaN,0051300000C41JZAAZ,0051B000009MC2dQAG,NaN,Inbound,Marketplace - SFDC AppExchange
3,3,0081300002OpToDAAV,0061300001ABzUgAAL,0051300000BAV3KAAX,9/22/2014 20:22,9/22/2014 20:22,100% Won / PO / Invoiced,4320.0,10,9/22/2014 20:22,...,0,Salesforce,NaN,User Adoption;User Productivity;Accessing CRM ...,Inside Sales,0051300000C41JZAAZ,0051B000009NLbZQAW,App Exchange,Inbound,Website Trial SignUp
4,4,0081300002OpYTKAA3,0061300001AC0orAAD,0051300000BAV3KAAX,9/23/2014 12:13,9/23/2014 12:13,0% probability Closed Expired,14250.0,10,9/23/2014 12:13,...,0,Salesforce,NaN,User Adoption;User Productivity;Accessing CRM ...,Field Sales,0051300000C41JZAAZ,0051B000009NLbZQAW,Website,Inbound,Website Contact Request
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34217,40990,008a000001t3lLuAAI,006a000000xh25QAAQ,00530000004M1PmAAK,1/11/2013 21:59,1/15/2013 17:32,0% probability Closed Lost,3000.0,50,1/11/2013 21:59,...,0,NaN,NaN,NaN,NaN,0051300000C41JZAAZ,0051B000009NLbZQAW,NaN,Inbound,Marketplace - SFDC AppExchange
34218,40991,008a000001t5F8AAAU,006a000000xiGgxAAE,00530000004M1PmAAK,1/18/2013 15:39,1/18/2013 15:39,0% probability Closed Lost,5000.0,100,1/18/2013 15:39,...,0,NaN,NaN,NaN,NaN,0051B000009NLbeQAG,0051B00000CsqZ4QAJ,NaN,Inbound,Marketplace - SFDC AppExchange
34219,40992,008a000001t5F9DAAU,006a000000xiGgxAAE,00530000004M1PmAAK,1/18/2013 15:39,1/18/2013 15:40,0% probability Closed Lost,5000.0,30,1/18/2013 15:39,...,0,NaN,NaN,NaN,NaN,0051B000009NLbeQAG,0051B00000CsqZ4QAJ,NaN,Inbound,Marketplace - SFDC AppExchange
34220,40993,008a000001t5FGiAAM,006a000000xiGkGAAU,00530000004M1PmAAK,1/18/2013 15:43,1/18/2013 15:43,0% probability Closed Lost,12000.0,70,1/18/2013 15:43,...,0,NaN,NaN,NaN,NaN,0051B000009NLbeQAG,0051B00000CsqZ4QAJ,NaN,NaN,NaN


In [130]:
numX = 4

In [150]:
def read_df_train(path):
    df = pd.read_csv(path, low_memory=False)
    df = df[['OpportunityId', 'CreatedDateForInsert', 'PrevOpportunityStageUpdate', 'Amount', 'Probability']]
    df['CreatedDateForInsert'] = pd.to_datetime(df['CreatedDateForInsert'],format='%m/%d/%Y %H:%M')
    df['PrevOpportunityStageUpdate'] = pd.to_datetime(df['PrevOpportunityStageUpdate'],format='%m/%d/%Y %H:%M')

    df = df.sort_values(by=['CreatedDateForInsert'])
    df['TimeDist'] = (df['CreatedDateForInsert'] - df['PrevOpportunityStageUpdate']).astype('timedelta64[m]')

    df = df.drop(['CreatedDateForInsert', 'PrevOpportunityStageUpdate'], axis=1)
    df = df.groupby('OpportunityId').agg(list)
    df['Amount'] = df['Amount'].apply(lambda x : x[0])
#     df['TimeDist'] = df['TimeDist'].apply(lambda x : [0.]*(numX-1) + x)
#     df['Probability'] = df['Probability'].apply(lambda x : [x[0]]*(numX-1) + x)
    return df


df = read_df_train('./data/train_data.csv')
df

,Amount,Probability,TimeDist
OpportunityId,,,
00613000019fCVIAA2,0.0,"[100, 0]","[0.0, 126831.0]"
0061300001AAxCTAA1,14250.0,"[100, 30, 0, 70, 70, 50, 70, 1, 60, 60, 60, 60...","[0.0, 1.0, 42989.0, 5633.0, 1732.0, 29116.0, 1..."
0061300001ABzUgAAL,4320.0,"[10, 30, 30, 30, 40, 40, 40, 40, 40, 90, 40, 9...","[0.0, 1094.0, 9015.0, 9016.0, 40238.0, 337.0, ..."
0061300001AC0orAAD,14250.0,"[10, 30, 30, 30, 30, 40, 0, 1, 1, 0, 0, 0]","[0.0, 144.0, 9011.0, 20334.0, 20349.0, 40239.0..."
0061300001AC0pGAAT,15000.0,"[10, 30, 30, 30, 40, 0, 1, 0, 0, 0]","[0.0, 141.0, 13070.0, 21711.0, 40240.0, 13135...."
...,...,...,...
006a000000xh25QAAQ,3000.0,"[100, 50, 0]","[0.0, 5493.0, 891341.0]"
006a000000xhhIZAAY,5000.0,"[30, 0]","[0.0, 891345.0]"
006a000000xiGgxAAE,5000.0,"[100, 30, 0]","[0.0, 1.0, 887133.0]"


In [151]:
def create_X_y(df):
    X = []
    y = []
    global numX
    for _, row in df.iterrows():
    #     print(row)
        price, probability, timeDist  = row
        for i in range(0, len(probability)-numX):
            y.append(probability[i+numX])
            X.append([probability[i+j] for j in range(numX)] + [timeDist[i+j] for j in range(1, numX+1)] + [price])
#             X.append([probability[i+j] for j in range(numX)] + [timeDist[i+j] for j in range(1, numX+1)])
            
#             X.append([probability[i+j] for j in range(numX)])

    y = np.array(y)
    X = np.array(X)
    return X, y


X, y = create_X_y(df)
# print(X.shape)
# print(y.shape)


reg = LinearRegression().fit(X, y)

print(f'score              = {reg.score(X, y)}')
print(f'mean_squared_error = {mean_squared_error(reg.predict(X), y)}')

score              = 0.6145931574719361
mean_squared_error = 521.2335465983272


In [35]:
soso = pd.read_csv('data/test_data_full.csv')
# soso2 = pd.read_csv('data/test_data_check_template.csv')

soso.shape, soso2.shape


NameError: name 'soso2' is not defined

In [149]:
def read_df_test(path):
    df = pd.read_csv(path, low_memory=False)
    df = df[['OpportunityId', 'CreatedDateForInsert', 'PrevOpportunityStageUpdate', 'Amount', 'Probability']]
    df['CreatedDateForInsert'] = pd.to_datetime(df['CreatedDateForInsert'],format='%Y-%m-%d %H:%M:%S')
    df['PrevOpportunityStageUpdate'] = pd.to_datetime(df['PrevOpportunityStageUpdate'],format='%m/%d/%Y %H:%M')

    df = df.sort_values(by=['CreatedDateForInsert'])
    df['TimeDist'] = (df['CreatedDateForInsert'] - df['PrevOpportunityStageUpdate']).astype('timedelta64[m]')

    df = df.drop(['CreatedDateForInsert', 'PrevOpportunityStageUpdate'], axis=1)
    df = df.groupby('OpportunityId').agg(list)
    df['Amount'] = df['Amount'].apply(lambda x : x[0])
    df['TimeDist'] = df['TimeDist'].apply(lambda x : [0.]*(numX-1) + x)
    df['Probability'] = df['Probability'].apply(lambda x : [x[0]]*(numX-1) + x)
    return df

df_test = read_df_test('data/test_data_full.csv')
df_test

,Amount,Probability,TimeDist
OpportunityId,,,
0061300001AC2HQAA1,81000.0,"[10, 10, 10, 10, 10, 10, 1, 10, 30, 10, 30, 20...","[0.0, 0.0, 0.0, 0.0, 8862.0, 8867.0, 9903.0, 1..."
0061300001ACPs0AAH,2270000.0,"[50, 50, 50, 50, 50, 5, 1, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 31593.0, 129435.0, 34456...."
0061300001ACQIvAAP,880000.0,"[30, 30, 30, 30, 30, 30, 30, 0]","[0.0, 0.0, 0.0, 0.0, 32828.0, 41508.0, 112595...."
0061300001AChYSAA1,76500.0,"[1, 1, 1, 1, 1, 30, 30, 20, 20, 20, 10, 10, 25...","[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1531.0, 18554.0..."
0061300001ADRAxAAP,0.0,"[1, 1, 1, 1, 40, 40, 10, 0, 1, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 2.0, 7075.0, 37741.0, 886..."
...,...,...,...
006a000000vWvHvAAK,7500.0,"[0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 60506.0, 970547.0]"
006a000000wfuZOAAY,20000.0,"[0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 978800.0]"
006a000000x9GNFAA2,20000.0,"[30, 30, 30, 30, 30, 30, 0]","[0.0, 0.0, 0.0, 0.0, 1.0, 132.0, 58889.0]"


In [153]:
def create_X_y_test(df):
    X = []
    y = []
    global numX
    for _, row in df.iterrows():
    #     print(row)
        price, probability, timeDist  = row
        for i in range(0, len(probability)-numX):
            y.append(probability[i+numX])
            X.append([probability[i+j] for j in range(numX)] + [timeDist[i+j] for j in range(1, numX+1)] + [price])
#             X.append([probability[i+j] for j in range(numX)] + [timeDist[i+j] for j in range(1, numX+1)])
            
#             X.append([probability[i+j] for j in range(numX)])

    y = np.array(y)
    X = np.array(X)
    return X, y

        

X_test, y_test = create_X_y_test(df_test)

print(X_test.shape)
print(y_test.shape)

(3702, 9)
(3702,)


In [156]:
# reg.score(X_test, y_test)

print(f'score              = {reg.score(X_test, y_test)}')
print(f'mean_squared_error = {mean_squared_error(reg.predict(X_test), y_test)}')

score              = 0.3590030945535382
mean_squared_error = 926.6479723159468
